# Integrated Stacking Model

When using neural networks as sub-models, it may be desirable to use a neural network as a meta-learner. In this model, we used the last layer of the three MODNet sub-models (trained on the PBE, HSE and experimental datasets) as inputs for our meta-learner.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models.vanilla import MODNetModel
from modnet.hyper_opt.fit_genetic import FitGenetic
import numpy as np
import pandas as pd
import os
import copy
from sklearn.model_selection import KFold

In [4]:
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

#### Loading the PBE, HSE and experimental datasets.

In [5]:
md_pbe = MODData.load('pbe_gap.zip')
md_pbe.df_targets.columns = ['gap']
md_hse = MODData.load('hse_gap.zip')
md_hse.df_targets.columns = ['gap']
md_exp = MODData.load('exp_gap_all')
md_exp.df_targets.columns = ['gap']
md_joint = MODData.load('exp_pbe_joint')
md_joint.df_targets = md_joint.df_targets.drop(columns=['pbe_gap', 'difference'])


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766

2021-06-18 00:54:31,864 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f5f2fac8f10> object, created with modnet version <=0.1.7
2021-06-18 00:54:34,273 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f5f2fac8e20> object, created with modnet version 0.1.8~develop
2021-06-18 00:54:38,637 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f5f2fac8fa0> object, created with modnet version <=0.1.7
2021-06-18 00:54:41,092 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f5f2fabf4c0> object,

#### MODNet model trained on the PBE dataset

In [21]:
k = 5
random_state = 202010
folds = MDKsplit(md_pbe,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    fpath = 'train-pbe_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
        train.df_targets.columns = ['gap']
    else:
        train.feature_selection(n=-1)
        train.save(fpath)
        
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    model1 = MODNetModel([[['gap']]],{'gap':1})
    model1.fit_preset(train,verbose=0)
    pred = model1.predict(test)
    true = test.df_targets
    error = pred-true
    error = error.drop(pred.index[((pred['gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print(mae)
    maes[i] = mae

NameError: name 'df_pbe' is not defined

In [95]:
#model1.save('MODNetModel1')

INFO:root:Saving model...
INFO:root:Saved model to MODNetModel1(.json/.h5/.pkl)


In [7]:
model1 = MODNetModel.load('MODNetModel1')

2021-06-18 00:54:42,964 - modnet - INFO - Loaded <modnet.models.ensemble.EnsembleMODNetModel object at 0x7f5f2fac85e0> object, created with modnet version 0.1.9


#### MODNet model trained on the HSE dataset

In [6]:
k = 5
random_state = 202010
folds = MDKsplit(md_hse,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    fpath = 'train-hse_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
        train.df_targets.columns = ['gap']
    else:
        train.feature_selection(n=-1)
        train.save(fpath)
        
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    model2 = MODNetModel([[['gap']]],{'gap':1})
    model2.fit_preset(train,verbose=0)
    pred = model2.predict(test)
    true = test.df_targets
    error = pred-true
    error = error.drop(pred.index[((pred['gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print(mae)
    maes[i] = mae

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f00accb70d0> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.333
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.394
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 

1.132328280405736


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f00a86986d0> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.358
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.401
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.351
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.385
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.370
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.382
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.343
INFO:root:Training preset #8/1

0.9613473074160188


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f00ace63670> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.387
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.430
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.370
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.374
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.400
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.389
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.352
INFO:root:Training preset #8/1

0.9850014001033152


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f00ad7f0430> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.357
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.379
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.348
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.392
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.374
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.388
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.388
INFO:root:Training preset #8/1

1.0600857373613162


INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f00b28dc250> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.433
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.461
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 7.480
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.446
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.427
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.414
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.413
INFO:root:Training preset #8/1

1.0738566065837396


In [96]:
#model2.save('MODNetModel2')

INFO:root:Saving model...
INFO:root:Saved model to MODNetModel2(.json/.h5/.pkl)


In [8]:
model2 = MODNetModel.load('MODNetModel2')

2021-06-18 00:54:43,364 - modnet - INFO - Loaded <modnet.models.ensemble.EnsembleMODNetModel object at 0x7f5e978611c0> object, created with modnet version 0.1.9


### Integrated Stacking Model (3 meta-learner architectures are shown)

In [13]:
# stacked generalization with linear meta model on blobs dataset
import pandas as pd
from sklearn.metrics import accuracy_score

In [14]:
# load models
def load_all_models():
    all_models = list()
    all_models = [model1, model2, model3]
    return all_models

In [15]:
# create stacked model input dataset as outputs from the ensemble

from keras import backend as K

def stacked_dataset(members, inputX):
    layer_output = np.zeros(len(inputX.df_targets))
    for model in members:
        layers = None
        for i in range(5):
            x = inputX.get_featurized_df().replace([np.inf, -np.inf, np.nan], 0)[
                model.model[i].optimal_descriptors[:model.model[i].n_feat]
            ].values

            # Scale the input features:
            x = np.nan_to_num(x)
            if model.model[i]._scaler is not None:
                x = model.model[i]._scaler.transform(x)
                x = np.nan_to_num(x)
            get_layer_output = K.function([model.model[i].model.layers[0].input],
                                  [model.model[i].model.layers[-2].output])
            layer = get_layer_output([x])[0]
            if layers is None:
                layers = layer
            else:
                layers = np.dstack((layers, layer))
        layer_output = np.mean(layers, axis=2)
    return layer_output

#### 1) Keras model with two hidden layers: 10 and 3 neurons respectively

In [12]:
# create the meta-learner model
import tensorflow as tf

def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(3, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
                loss='mse',
                metrics=['accuracy'])
    return model

In [13]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # fit standalone model
    model = get_compiled_model()
    model.fit(stackedX, inputy, epochs=500, verbose=0)
    return model

In [14]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [15]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
    
        
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    ga = FitGenetic(train)
    model3 = ga.run(size_pop=20, num_generations=10, n_jobs=10)
    
    # load all models
    members = [model1, model2, model3]
    print('Loaded %d models' % len(members))
    
    # fit stacked model on test dataset
    model = fit_stacked_model(members, inputX=train, inputy=train.df_targets['exp_gap'])

    # evaluate model on test set
    pred = stacked_prediction(members, model, test)
    pred = pd.DataFrame(pred)
    pred = pred.rename(columns={pred.columns[0]: 'exp_gap'})
    true = test.df_targets
    for j in range(len(pred)):
        pred = pred.rename(index={pred.index[j]: true.index[j]})
    error = pred-true
    error = error['exp_gap'].drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('MAE =')
    print(mae)
    maes[i] = mae

2021-06-14 18:07:15,431 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:07:15,491 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:07:15,516 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:07:15,597 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:07:15,626 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:07:15,693 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:07:15,718 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:07:15,770 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:07:15,791 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:07:15,875 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 18:07:16,381 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f7d0a249e20> object, created with modnet version 0.1.9
2021-06-14 18

100%|██████████| 100/100 [27:17<00:00, 16.37s/it]

2021-06-14 18:34:37,858 - modnet - INFO - Loss per individual: ind 0: 0.606 	ind 1: 0.633 	ind 2: 0.667 	ind 3: 0.655 	ind 4: 0.671 	ind 5: 0.666 	ind 6: 0.626 	ind 7: 0.668 	ind 8: 0.665 	ind 9: 0.693 	ind 10: 0.595 	ind 11: 0.657 	ind 12: 0.631 	ind 13: 0.650 	ind 14: 0.622 	ind 15: 0.594 	ind 16: 0.770 	ind 17: 0.718 	ind 18: 0.659 	ind 19: 0.672 	


2021-06-14 18:34:41,306 - modnet - INFO - Generation number 1
2021-06-14 18:34:49,477 - modnet - INFO - Multiprocessing on 10 cores. Total of 48 cores available.


100%|██████████| 100/100 [27:31<00:00, 16.52s/it] 

...

2021-06-15 11:52:22,513 - modnet - INFO - Loss per individual: ind 0: 0.627 	ind 1: 0.642 	ind 2: 0.760 	ind 3: 0.639 	ind 4: 0.687 	ind 5: 0.642 	ind 6: 0.668 	ind 7: 0.596 	ind 8: 0.648 	ind 9: 0.660 	ind 10: 0.613 	ind 11: 0.643 	ind 12: 0.673 	ind 13: 0.639 	ind 14: 0.644 	ind 15: 0.665 	ind 16: 0.608 	ind 17: 0.624 	ind 18: 0.737 	ind 19: 0.655 	


2021-06-15 11:52:24,475 - modnet - INFO - Generation number 6
2021-06-15 11:52:32,226 - modnet - INFO - Multiprocessing on 10 cores. Total of 48 cores available.


100%|██████████| 100/100 [33:55<00:00, 20.35s/it] 

2021-06-15 12:26:27,637 - modnet - INFO - Loss per individual: ind 0: 0.578 	ind 1: 0.657 	ind 2: 0.602 	ind 3: 0.649 	ind 4: 0.605 	ind 5: 0.593 	ind 6: 0.591 	ind 7: 0.643 	ind 8: 0.693 	ind 9: 0.622 	ind 10: 0.617 	ind 11: 0.620 	ind 12: 0.594 	ind 13: 0.582 	ind 14: 0.696 	ind 15: 0.593 	ind 16: 0.677 	ind 17: 0.643 	ind 18: 0.645 	ind 19: 0.610 	


2021-06-15 12:26:30,556 - modnet - INFO - Early stopping: same best model for 4 consecutive generations
Loaded 3 models
MAE =
0.3719301076520654


In [16]:
maes

array([0.37659606, 0.36093865, 1.17993554, 0.31894951, 0.37193011])

In [17]:
maes.mean()

0.521669973444622

#### 2) Keras model with one hidden layer of 3 neurons

In [9]:
# create the meta-learner model
import tensorflow as tf

def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(3, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
                loss='mse',
                metrics=['accuracy'])
    return model

In [10]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # fit standalone model
    model = get_compiled_model()
    model.fit(stackedX, inputy, epochs=500, verbose=0)
    return model

In [11]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat= model.predict(stackedX)
    return yhat

In [16]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
    
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    ga = FitGenetic(train)
    model3 = ga.run(size_pop=20, num_generations=10, n_jobs=32)
    
    # load all models
    members = [model1, model2, model3]
    print('Loaded %d models' % len(members))
    
    # fit stacked model on test dataset
    model = fit_stacked_model(members, inputX=train, inputy=train.df_targets.to_numpy())

    # evaluate model on test set
    pred= stacked_prediction(members, model, test)
    pred = pd.DataFrame(pred)
    pred = pred.rename(columns={pred.columns[0]: 'exp_gap'})
    true = test.df_targets
    for j in range(len(pred)):
        pred = pred.rename(index={pred.index[j]: true.index[j]})
    error = pred-true
    error = error['exp_gap'].drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('MAE =')
    print(mae)
    maes[i] = mae

2021-06-18 09:25:04,411 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 09:25:04,435 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 09:25:04,447 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 09:25:04,464 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 09:25:04,475 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 09:25:04,490 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 09:25:04,499 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 09:25:04,513 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 09:25:04,523 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 09:25:04,538 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 09:25:04,800 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f5e9786e280> object, created with modnet version 0.1.9
2021-06-18 09

100%|██████████| 100/100 [07:30<00:00,  4.51s/it]

2021-06-18 09:32:40,965 - modnet - INFO - Loss per individual: ind 0: 0.614 	ind 1: 0.660 	ind 2: 0.612 	ind 3: 0.683 	ind 4: 0.724 	ind 5: 0.692 	ind 6: 0.602 	ind 7: 0.626 	ind 8: 0.668 	ind 9: 0.698 	ind 10: 0.654 	ind 11: 0.615 	ind 12: 0.598 	ind 13: 0.685 	ind 14: 0.665 	ind 15: 0.579 	ind 16: 0.644 	ind 17: 0.638 	ind 18: 0.644 	ind 19: 0.714 	


2021-06-18 09:32:43,215 - modnet - INFO - Generation number 1
2021-06-18 09:32:49,022 - modnet - INFO - Multiprocessing on 32 cores. Total of 128 cores available.


100%|██████████| 100/100 [06:29<00:00,  3.89s/it]

...

2021-06-18 13:10:44,990 - modnet - INFO - Loss per individual: ind 0: 0.617 	ind 1: 0.607 	ind 2: 0.596 	ind 3: 0.642 	ind 4: 0.617 	ind 5: 0.634 	ind 6: 0.649 	ind 7: 0.612 	ind 8: 0.591 	ind 9: 1.089 	ind 10: 1.013 	ind 11: 0.656 	ind 12: 0.629 	ind 13: 0.603 	ind 14: 0.607 	ind 15: 0.650 	ind 16: 0.635 	ind 17: 0.636 	ind 18: 0.611 	ind 19: 0.616 	


2021-06-18 13:10:47,444 - modnet - INFO - Generation number 5
2021-06-18 13:10:55,251 - modnet - INFO - Multiprocessing on 32 cores. Total of 128 cores available.


100%|██████████| 100/100 [09:33<00:00,  5.74s/it]

2021-06-18 13:20:29,195 - modnet - INFO - Loss per individual: ind 0: 0.603 	ind 1: 0.646 	ind 2: 0.612 	ind 3: 0.594 	ind 4: 0.628 	ind 5: 0.633 	ind 6: 0.634 	ind 7: 0.608 	ind 8: 0.605 	ind 9: 0.601 	ind 10: 0.621 	ind 11: 0.615 	ind 12: 0.618 	ind 13: 0.630 	ind 14: 0.594 	ind 15: 0.604 	ind 16: 0.646 	ind 17: 0.622 	ind 18: 0.650 	ind 19: 0.621 	


2021-06-18 13:20:31,562 - modnet - INFO - Early stopping: same best model for 4 consecutive generations
Loaded 3 models
MAE =
0.4065135907032914


In [17]:
maes

array([0.41552443, 0.39190951, 0.32807523, 0.40762547, 0.40651359])

In [18]:
maes.mean()

0.38992964501971805

#### 3) Extra Trees Regressor scikit-learn method

In [19]:
from sklearn.ensemble import ExtraTreesRegressor

# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # fit standalone model
    model = ExtraTreesRegressor(n_estimators=500)
    model.fit(stackedX, inputy)
    return model

In [20]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [21]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
        
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    ga = FitGenetic(train)
    model3 = ga.run(size_pop=20, num_generations=10, n_jobs=32)
    
    # load all models
    members = [model1, model2, model3]
    print('Loaded %d models' % len(members))
    
    # fit stacked model on test dataset
    model = fit_stacked_model(members, inputX=train, inputy=train.df_targets.to_numpy())

    # evaluate model on test set
    pred= stacked_prediction(members, model, test)
    pred = pd.DataFrame(pred)
    pred = pred.rename(columns={pred.columns[0]: 'exp_gap'})
    true = test.df_targets
    for j in range(len(pred)):
        pred = pred.rename(index={pred.index[j]: true.index[j]})
    error = pred-true
    error = error['exp_gap'].drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('MAE =')
    print(mae)
    maes[i] = mae

2021-06-18 13:20:48,862 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 13:20:48,881 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 13:20:48,892 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 13:20:48,916 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 13:20:48,927 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 13:20:48,955 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 13:20:48,964 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 13:20:48,979 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 13:20:48,989 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 13:20:49,004 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-18 13:20:52,360 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f5e99403e80> object, created with modnet version 0.1.9
2021-06-18 13

100%|██████████| 100/100 [06:31<00:00,  3.92s/it]

2021-06-18 13:27:30,771 - modnet - INFO - Loss per individual: ind 0: 0.634 	ind 1: 0.646 	ind 2: 0.682 	ind 3: 0.698 	ind 4: 0.649 	ind 5: 0.611 	ind 6: 0.699 	ind 7: 0.630 	ind 8: 0.648 	ind 9: 0.650 	ind 10: 0.655 	ind 11: 0.632 	ind 12: 0.626 	ind 13: 0.736 	ind 14: 0.722 	ind 15: 0.696 	ind 16: 0.637 	ind 17: 0.637 	ind 18: 0.655 	ind 19: 0.756 	


2021-06-18 13:27:33,104 - modnet - INFO - Generation number 1
2021-06-18 13:27:39,475 - modnet - INFO - Multiprocessing on 32 cores. Total of 128 cores available.


100%|██████████| 100/100 [06:43<00:00,  4.04s/it]

...

2021-06-18 17:40:13,122 - modnet - INFO - Loss per individual: ind 0: 0.636 	ind 1: 0.657 	ind 2: 0.612 	ind 3: 0.740 	ind 4: 0.692 	ind 5: 0.590 	ind 6: 0.659 	ind 7: 0.619 	ind 8: 0.669 	ind 9: 0.673 	ind 10: 0.597 	ind 11: 0.740 	ind 12: 0.666 	ind 13: 0.648 	ind 14: 0.633 	ind 15: 0.657 	ind 16: 0.601 	ind 17: 0.605 	ind 18: 0.580 	ind 19: 0.606 	


2021-06-18 17:40:15,468 - modnet - INFO - Generation number 6
2021-06-18 17:40:25,419 - modnet - INFO - Multiprocessing on 32 cores. Total of 128 cores available.


100%|██████████| 100/100 [08:41<00:00,  5.21s/it]

2021-06-18 17:49:06,528 - modnet - INFO - Loss per individual: ind 0: 0.676 	ind 1: 0.643 	ind 2: 0.632 	ind 3: 0.616 	ind 4: 0.654 	ind 5: 0.597 	ind 6: 0.599 	ind 7: 0.677 	ind 8: 0.627 	ind 9: 0.616 	ind 10: 0.635 	ind 11: 0.684 	ind 12: 0.619 	ind 13: 0.628 	ind 14: 0.620 	ind 15: 0.620 	ind 16: 0.611 	ind 17: 0.617 	ind 18: 0.631 	ind 19: 0.648 	


2021-06-18 17:49:08,899 - modnet - INFO - Early stopping: same best model for 4 consecutive generations
Loaded 3 models
MAE =
0.3765824556855847


In [22]:
maes

array([0.37545936, 0.36791082, 0.37724455, 0.35054561, 0.37658246])

In [23]:
maes.mean()

0.3695485613540711

#### Conclusion

Relative percentage change of the error compared to MODNet_exp_GA reference: -7%